In [2]:
import pandas as pd
import numpy as np
import talib


In [ ]:
def custom_trading_strategy(df, period=878, mode=1, invert_signal=1, filter1=41, filter2=0):
    """
    Custom trading strategy implementation based on the provided TradingView logic.
    
    Args:
        df (pd.DataFrame): A dataframe with columns 'date', 'open', 'high', 'low', 'close', 'volume'.
        period (int): The period for calculating the summation (similar to SMA in Pine Script).
        mode (int): The trading mode (1, 2, 3, or 4) to determine entry and exit conditions.
        invert_signal (int): Signal inversion (1 for normal, -1 for inverse).
        filter1 (float): The first filter threshold for the green condition.
        filter2 (float): The second filter threshold for the red condition.
    
    Returns:
        pd.DataFrame: A dataframe containing the original data with buy/sell signals appended.
    """

    # Calculate body and bodyVar (difference between current and previous body)
    df['body'] = df['close'] - df['open']
    df['bodyVar'] = df['body'].diff()

    # SumVar is similar to a moving average of bodyVar
    df['sumVar'] = talib.SMA(df['bodyVar'], timeperiod=period)

    # Initialize green and red columns
    df['green'] = np.where(df['sumVar'] > filter1, df['sumVar'], np.nan)
    df['red'] = np.where(df['sumVar'] < -filter2, df['sumVar'], np.nan)

    # Initialize signals for c1 and c2 based on trading mode
    df['c1'] = np.nan
    df['c2'] = np.nan

    if mode == 1:
        df['c1'] = df['red'] < df['red'].shift(1)
        df['c2'] = df['green'] > df['green'].shift(1)
    elif mode == 2:
        df['c1'] = df['red'] > df['red'].shift(1)
        df['c2'] = df['green'] < df['green'].shift(1)
    elif mode == 3:
        df['c1'] = df['red'] < df['red'].shift(1)
        df['c2'] = df['green'] < df['green'].shift(1)
    elif mode == 4:
        df['c1'] = df['red'] > df['red'].shift(1)
        df['c2'] = df['green'] > df['green'].shift(1)

    # Generate buy/sell signals based on c1 and c2
    df['signal'] = np.nan
    df['signal'] = np.where(df['c1'], 1 * invert_signal, df['signal'])
    df['signal'] = np.where(df['c2'], -1 * invert_signal, df['signal'])

    # Trading logic: Create buy/sell signals
    df['position'] = df['signal'].shift(1)
    df['buy'] = np.where(df['position'] > 0, 1, 0)  # Buy signal
    df['sell'] = np.where(df['position'] < 0, 1, 0)  # Sell signal

    return df

In [10]:
df = pd.read_csv('../NDX5years.csv')
df.columns = ['date', 'open', 'high', 'low', 'close', 'volume']

In [11]:
df.dtypes

date       object
open      float64
high      float64
low       float64
close     float64
volume      int64
dtype: object

In [12]:
period=878
mode=1
invert_signal=1
filter1=41
filter2=0

In [13]:
# Calculate body and bodyVar (difference between current and previous body)
df['body'] = df['close'] - df['open']
df['bodyVar'] = df['body'].diff()

In [14]:
df

,date,open,high,low,close,volume,body,bodyVar
0,2019-10-25,7926.6001,8033.2900,7926.6001,8029.2202,1911580000,102.6201,NaN
1,2019-10-28,8071.3901,8119.7300,8070.8999,8110.6699,1959920000,39.2798,-63.3403
2,2019-10-29,8101.5498,8108.6099,8045.6401,8047.5098,1842720000,-54.0400,-93.3198
3,2019-10-30,8056.8999,8096.3301,8017.9199,8083.1099,1936060000,26.2100,80.2500
4,2019-10-31,8100.6201,8110.4399,8041.6499,8083.8301,2276140000,-16.7900,-43.0000
...,...,...,...,...,...,...,...,...
1244,2024-10-07,19954.6602,19990.1191,19763.0391,19800.7402,5240320000,-153.9200,-163.7793
1245,2024-10-08,19902.2109,20132.6699,19881.0801,20107.7793,5566510000,205.5684,359.4884
1246,2024-10-09,20097.5605,20284.4199,20047.8594,20268.8594,5228670000,171.2989,-34.2695
1247,2024-10-10,20165.2500,20314.2402,20117.1992,20241.7598,5855390000,76.5098,-94.7891


In [ ]:
#def custom_trading_strategy(df, period=878, mode=1, invert_signal=1, filter1=41, filter2=0):




# SumVar is similar to a moving average of bodyVar
df['sumVar'] = talib.SMA(df['bodyVar'], timeperiod=period)

# Initialize green and red columns
df['green'] = np.where(df['sumVar'] > filter1, df['sumVar'], np.nan)
df['red'] = np.where(df['sumVar'] < -filter2, df['sumVar'], np.nan)

# Initialize signals for c1 and c2 based on trading mode
df['c1'] = np.nan
df['c2'] = np.nan

if mode == 1:
    df['c1'] = df['red'] < df['red'].shift(1)
    df['c2'] = df['green'] > df['green'].shift(1)
elif mode == 2:
    df['c1'] = df['red'] > df['red'].shift(1)
    df['c2'] = df['green'] < df['green'].shift(1)
elif mode == 3:
    df['c1'] = df['red'] < df['red'].shift(1)
    df['c2'] = df['green'] < df['green'].shift(1)
elif mode == 4:
    df['c1'] = df['red'] > df['red'].shift(1)
    df['c2'] = df['green'] > df['green'].shift(1)

# Generate buy/sell signals based on c1 and c2
df['signal'] = np.nan
df['signal'] = np.where(df['c1'], 1 * invert_signal, df['signal'])
df['signal'] = np.where(df['c2'], -1 * invert_signal, df['signal'])

# Trading logic: Create buy/sell signals
df['position'] = df['signal'].shift(1)
df['buy'] = np.where(df['position'] > 0, 1, 0)  # Buy signal
df['sell'] = np.where(df['position'] < 0, 1, 0)  # Sell signal
